# 🔌 Model Context Protocol (MCP) 기초

## 📚 학습 목표
MCP(Model Context Protocol)는 2024년 11월 Anthropic에서 발표한 혁신적인 프로토콜입니다!

### 🎯 배울 내용
- **MCP 개념**: 왜 필요하고 어떤 문제를 해결하는가?
- **LangChain MCP Adapters**: 실제 도구와 연결하는 방법
- **MCP 서버 만들기**: FastMCP로 나만의 도구 서버 구축
- **실전 적용**: 계산기 + 텍스트 처리 도구 연동

### 🌟 특징
- **2024년 최신 기술**: AI 개발의 새로운 표준
- **실제 동작하는 코드**: 모든 예제가 실제 실행 가능
- **점진적 학습**: 기초부터 실전까지 단계별 진행

# 1. 📦 라이브러리 설치

**MCP와 LangChain**을 연동하기 위한 최신 라이브러리들을 설치합니다.

### 설치할 도구들
- **langchain-mcp-adapters**: LangChain과 MCP 연결
- **fastmcp**: 간단한 MCP 서버 구축 도구
- **mcp**: MCP 프로토콜 라이브러리

In [ ]:
# MCP 관련 라이브러리 설치

print("📦 MCP 및 관련 라이브러리를 설치하고 있습니다...")

import subprocess
import sys

# MCP 핵심 라이브러리들
mcp_packages = [
    "langchain-mcp-adapters",
    "fastmcp", 
    "mcp",
    "langchain",
    "langgraph",
    "langchain-huggingface",
    "transformers",
    "torch",
    "langchain-teddynote"
]

for package in mcp_packages:
    try:
        subprocess.run([sys.executable, "-m", "pip", "install", "-q", package], 
                      check=True, capture_output=True)
        print(f"✅ {package}")
    except subprocess.CalledProcessError:
        print(f"⚠️ {package} 설치 실패 - 수동 설치 필요")
    except Exception as e:
        print(f"⚠️ {package} 설치 중 오류: {str(e)}")

print("\n✅ 설치 완료! 이제 MCP 프로토콜을 배워보겠습니다.")

# 2. 🤔 MCP가 무엇인가?

## Model Context Protocol의 혁신

### 🚨 **기존 문제점**
AI 모델이 외부 도구(API, 데이터베이스, 파일 시스템 등)와 연결하려면:
- 각 도구마다 **다른 연결 방식**
- **복잡한 설정**과 **반복적인 코드**
- **표준화 부족**으로 유지보수 어려움

### ✨ **MCP의 해결책**
```
AI 모델 ↔ [MCP Protocol] ↔ 모든 도구들
```

- **표준화된 인터페이스**: 하나의 방식으로 모든 도구 연결
- **플러그인 생태계**: 누구나 MCP 서버를 만들어 공유
- **간편한 연결**: 복잡한 설정 없이 즉시 사용

### 🌟 **실제 예시**

#### Before MCP:
```python
# 각 도구마다 다른 방식
calculator = CalculatorTool(api_key="...", endpoint="...")
weather = WeatherAPI(token="...", format="json")
database = DatabaseTool(connection_string="...")
```

#### After MCP:
```python
# 모든 도구를 동일한 방식으로!
mcp_client = MCPClient()
tools = mcp_client.get_tools(["calculator", "weather", "database"])
```

### 🚀 **MCP의 핵심 장점**
1. **🔌 플러그 앤 플레이**: 설정 없이 즉시 연결
2. **🌐 에코시스템**: 전 세계 개발자들이 만든 도구 활용
3. **🛡️ 보안**: 표준화된 권한 관리
4. **⚡ 성능**: 효율적인 통신 프로토콜

In [ ]:
# 필수 라이브러리 import

import warnings
warnings.filterwarnings('ignore')

# FastMCP 직접 사용 (Colab 환경에서 설치 보장)
from fastmcp import FastMCP
print("✅ FastMCP 직접 사용 - Colab 환경에서 설치 보장")

# 기타 유틸리티 (MCP 서버 구현용)
import os
import json
import time
from typing import Dict, Any, List

print("✅ MCP 학습을 위한 핵심 라이브러리 로드 완료!")

# MCP의 핵심 개념 시각화
print("\n🌟 FastMCP 아키텍처:")
print("   AI Client ← → FastMCP Protocol ← → 실제 도구들")
print("   (Claude Desktop)  (MCP 표준 인터페이스)  (계산기, 텍스트 처리 등)")

# 3. 🛠️ 첫 번째 MCP 서버 만들기

**FastMCP**를 사용해서 간단한 계산기 MCP 서버를 만들어보겠습니다.

### 🎯 만들 도구들
1. **add**: 두 수를 더하는 도구
2. **multiply**: 두 수를 곱하는 도구
3. **calculate**: 복잡한 수식을 계산하는 도구

### 🌟 FastMCP의 장점
- **간단한 문법**: 파이썬 함수로 쉽게 도구 정의
- **자동 타입 검증**: 입력 타입을 자동으로 확인
- **즉시 실행**: 별도 서버 없이 바로 테스트 가능

In [ ]:
# FastMCP 계산기 서버 구현

# FastMCP 계산기 서버 생성
calculator_mcp = FastMCP("Calculator")

@calculator_mcp.tool()
def add(a: float, b: float) -> float:
    """두 개의 숫자를 더하여 합계를 반환합니다.
    
    Args:
        a (float): 첫 번째 숫자 (정수 또는 소수)
        b (float): 두 번째 숫자 (정수 또는 소수)
    
    Returns:
        float: 두 숫자의 합계
        
    Examples:
        - add(5, 3) → 8.0
        - add(2.5, 1.7) → 4.2
        - add(-10, 15) → 5.0
    
    Use cases:
        - 간단한 산술 계산
        - 재정 계산의 기초 연산
        - 데이터 분석에서 수치 합산
    """
    result = a + b
    print(f"🔢 [FastMCP Calculator] 덧셈: {a} + {b} = {result}")
    return result

@calculator_mcp.tool()
def multiply(a: float, b: float) -> float:
    """두 개의 숫자를 곱하여 곱셈 결과를 반환합니다.
    
    Args:
        a (float): 첫 번째 숫자 (곱하는 수)
        b (float): 두 번째 숫자 (곱해지는 수)
    
    Returns:
        float: 두 숫자의 곱셈 결과
        
    Examples:
        - multiply(4, 7) → 28.0
        - multiply(2.5, 3) → 7.5
        - multiply(-2, 5) → -10.0
    
    Use cases:
        - 면적, 부피 계산
        - 비율 및 배율 계산
        - 단위 변환 (환율, 길이 등)
        - 통계 분석에서 가중 계산
    """
    result = a * b
    print(f"🔢 [FastMCP Calculator] 곱셈: {a} × {b} = {result}")
    return result

@calculator_mcp.tool()
def calculate(expression: str) -> str:
    """복잡한 수학 표현식을 계산하고 결과를 반환합니다.
    
    Args:
        expression (str): 계산할 수학 표현식 (사칙연산, 괄호 지원)
    
    Returns:
        str: 계산 결과 또는 오류 메시지
        
    Examples:
        - calculate("10 + 5 * 2") → "계산 결과: 10 + 5 * 2 = 20"
        - calculate("(15 - 3) / 4") → "계산 결과: (15 - 3) / 4 = 3.0"
        - calculate("2.5 * 4 + 1") → "계산 결과: 2.5 * 4 + 1 = 11.0"
    
    Use cases:
        - 복잡한 수식 계산
        - 공학 계산 및 과학 연산
        - 재무 계산 (이자, 수익률 등)
        - 통계 분석에서 복합 계산
        
    Note:
        보안상 안전한 문자만 허용 (0-9, +, -, *, /, ., (, ), %)
    """
    try:
        allowed_chars = "0123456789+-*/.() "
        if all(c in allowed_chars for c in expression):
            result = eval(expression)
            print(f"🔢 [FastMCP Calculator] 수식 계산: {expression} = {result}")
            return f"계산 결과: {expression} = {result}"
        else:
            return "오류: 허용되지 않는 문자가 포함되어 있습니다. (사용 가능: 0-9, +, -, *, /, ., (, ), %)"
    except Exception as e:
        error_msg = f"계산 오류: {str(e)}"
        print(f"❌ [FastMCP Calculator] {error_msg}")
        return error_msg

print("✅ FastMCP 계산기 서버 구현 완료!")

## 🤔 왜 FastMCP 구조를 보여주는가?

### 📚 교육 목적
- **MCP 표준 이해**: 실제 MCP 서버 구조와 동일한 패턴 학습
- **확장성 체험**: 로컬 환경에서 실제 서버로 쉽게 변환 가능
- **호환성 인식**: Claude Desktop, Cursor 등과 호환되는 표준 방식

### ⚠️ Colab 환경의 제약사항
- **네트워크 포트 바인딩 제한** (보안 정책)
- **별도 서버 프로세스 실행 불가**
- **외부 MCP 클라이언트 연결 차단**
- **💡 따라서 교육용으로는 In-Process 방식 사용!**

### 🚀 실제 환경에서는
- **🏠 로컬 컴퓨터**: 진짜 MCP 서버로 실행 가능
- **☁️ 클라우드**: Google Cloud Run, AWS Lambda 등 배포
- **🌍 실제 활용**: 여러 AI 앱이 동시에 접근 가능

# 4. 📝 텍스트 처리 MCP 서버 만들기

이번에는 **텍스트 처리** 도구들을 가진 또 다른 MCP 서버를 만들어보겠습니다.

### 🎯 만들 도구들
1. **count_words**: 텍스트의 단어 수 세기
2. **reverse_text**: 텍스트 뒤집기
3. **extract_keywords**: 핵심 키워드 추출

### 💡 실제 활용 사례
- **콘텐츠 분석**: 블로그 포스트, 보고서 분석
- **텍스트 가공**: 데이터 전처리, 정제
- **정보 추출**: 문서에서 핵심 정보 추출

In [ ]:
# FastMCP 텍스트 처리 서버 구현

# FastMCP 텍스트 처리 서버 생성
text_mcp = FastMCP("TextProcessor")

@text_mcp.tool()
def count_words(text: str) -> str:
    """텍스트의 단어 수, 문자 수, 줄 수를 세어서 상세한 분석 결과를 제공합니다.
    
    Args:
        text (str): 분석할 텍스트 (한국어, 영어, 특수문자 모두 지원)
    
    Returns:
        str: 텍스트 분석 결과 (단어 수, 문자 수, 줄 수 포함)
        
    Examples:
        - count_words("안녕하세요 AI입니다") → "단어 수: 2개, 문자 수: 9개, 줄 수: 1개"
        - count_words("Hello\nWorld") → "단어 수: 2개, 문자 수: 11개, 줄 수: 2개"
    
    Use cases:
        - 문서 길이 측정 및 분석
        - 블로그 포스트, 기사 글자 수 확인
        - 소셜미디어 콘텐츠 분석
        - 학술 논문 단어 수 체크
        - 번역 작업 전 텍스트 분량 파악
        
    Note:
        공백으로 구분된 문자열을 단어로 인식하며, 한글과 영어 모두 정확히 카운트합니다.
    """
    words = len(text.split())
    characters = len(text)
    lines = len(text.split('\n'))
    
    result = f"📊 텍스트 분석 결과:\n단어 수: {words}개\n문자 수: {characters}개\n줄 수: {lines}개"
    print(f"📊 [FastMCP TextProcessor] 텍스트 분석: {words}단어, {characters}문자, {lines}줄")
    return result

@text_mcp.tool()
def reverse_text(text: str) -> str:
    """텍스트의 문자 순서를 거꾸로 뒤집어서 반환합니다.
    
    Args:
        text (str): 뒤집을 텍스트 (모든 언어와 특수문자 지원)
    
    Returns:
        str: 문자 순서가 뒤집힌 텍스트
        
    Examples:
        - reverse_text("Hello") → "olleH"
        - reverse_text("안녕하세요") → "요세하녕안"
        - reverse_text("ABC123") → "321CBA"
    
    Use cases:
        - 암호화/복호화 기초 작업
        - 텍스트 패턴 분석
        - 회문(palindrome) 확인 도구
        - 창의적 텍스트 변형
        - 데이터 전처리에서 텍스트 변환
        - 언어학 연구 및 분석
        
    Note:
        유니코드 문자를 포함한 모든 문자를 정확히 뒤집습니다.
    """
    reversed_text = text[::-1]
    print(f"🔄 [FastMCP TextProcessor] 텍스트 뒤집기: '{text}' → '{reversed_text}'")
    return f"뒤집은 결과: {reversed_text}"

@text_mcp.tool()
def extract_keywords(text: str, top_k: int = 5) -> str:
    """텍스트에서 가장 중요한 키워드들을 빈도 기반으로 추출합니다.
    
    Args:
        text (str): 키워드를 추출할 텍스트
        top_k (int): 추출할 키워드 개수 (기본값: 5개)
    
    Returns:
        str: 추출된 키워드 목록 (빈도순으로 정렬)
        
    Examples:
        - extract_keywords("AI는 미래 기술입니다. AI 발전이 중요합니다.") 
          → "추출된 키워드: AI, 기술, 미래, 발전, 중요"
        - extract_keywords("Python programming language", 3) 
          → "추출된 키워드: python, programming, language"
    
    Use cases:
        - 문서 요약 및 핵심 내용 파악
        - 검색 엔진 최적화(SEO) 키워드 분석
        - 소셜미디어 트렌드 분석
        - 학술 논문 주제어 추출
        - 뉴스 기사 태그 생성
        - 마케팅 콘텐츠 분석
        
    Algorithm:
        1. 텍스트 소문자 변환 및 단어 분리
        2. 불용어(조사, 전치사 등) 제거
        3. 2글자 이하 단어 필터링
        4. 빈도 계산 후 상위 키워드 추출
        
    Note:
        한국어와 영어 불용어를 자동으로 제거하여 의미있는 키워드만 추출합니다.
    """
    import re
    from collections import Counter
    
    # 불용어 (실제로는 더 많은 불용어 사용)
    stop_words = {'의', '가', '이', '은', '는', '을', '를', '에', '와', '과', '로', '으로', 
                  '에서', 'and', 'or', 'but', 'in', 'on', 'at', 'to', 'for', 'of', 'with'}
    
    # 텍스트 전처리
    words = re.findall(r'\b\w+\b', text.lower())
    filtered_words = [word for word in words if word not in stop_words and len(word) > 2]
    
    # 빈도 계산 및 상위 키워드 추출
    word_freq = Counter(filtered_words)
    keywords = [word for word, freq in word_freq.most_common(top_k)]
    
    result = f"🔍 추출된 키워드 (상위 {top_k}개): {', '.join(keywords)}"
    print(f"🔍 [FastMCP TextProcessor] 키워드 추출: {keywords}")
    return result

print("✅ FastMCP 텍스트 처리 서버 구현 완료!")

# 5. 🏠 로컬 환경에서 실제 MCP 서버 실행하기

## 🔌 MCP 통신 방식 이해하기

### 📡 MCP 프로토콜의 3가지 통신 방식

| 통신 방식 | 용도 | 장점 |
|-----------|------|------|
| **stdio** | 로컬 개발, Claude Desktop | 간단한 설정, 빠른 실행 |
| **HTTP** | 클라우드 배포, 여러 클라이언트 | 웹 표준, 확장성 |
| **WebSocket** | 실시간 기능 | 실시간 통신 |

### 🎯 시나리오별 선택 가이드

```
🏠 로컬 개발/테스트        → stdio
☁️ 클라우드 배포          → HTTP
⚡ 실시간 기능 필요       → WebSocket
```

## 📝 1단계: MCP 서버 파일 생성

### calculator_server.py 파일 생성:
```python
# calculator_server.py
from fastmcp import FastMCP

calculator_server = FastMCP("Calculator")

@calculator_server.tool()
def add(a: float, b: float) -> float:
    """두 개의 숫자를 더합니다."""
    return a + b

@calculator_server.tool()
def multiply(a: float, b: float) -> float:
    """두 개의 숫자를 곱합니다."""
    return a * b

if __name__ == "__main__":
    calculator_server.run()
```

## 🚀 2단계: MCP 서버 실행

### stdio 방식 (로컬 개발용)
```bash
# 서버 실행
python -m fastmcp serve calculator_server.py
```

### HTTP 방식 (배포용)
```bash
# HTTP 서버 실행
python -m fastmcp serve calculator_server.py --transport http --port 8080

# 상태 확인
curl http://127.0.0.1:8080/health
```

## 🔌 3단계: 클라이언트에서 연결

### Python 클라이언트 예시:
```python
from langchain_mcp_adapters import MCPClientToolkit

# stdio 방식 연결
toolkit = MCPClientToolkit(
    server_command=["python", "-m", "fastmcp", "serve", "calculator_server.py"]
)

# 도구 가져오기
tools = toolkit.get_tools()

# AI Agent에 연결
from langgraph.prebuilt import create_react_agent
agent = create_react_agent(model=llm, tools=tools)
```

### Claude Desktop 연결:
```json
// ~/.claude_desktop_config.json
{
  "mcpServers": {
    "calculator": {
      "command": "python",
      "args": ["-m", "fastmcp", "serve", "/path/to/calculator_server.py"]
    }
  }
}
```

## ⚡ 핵심 포인트

| 특징 | 현재 노트북 (교육용) | 실제 MCP (로컬) |
|------|---------------------|----------------|
| **서버 실행** | In-Process | 독립 프로세스 |
| **통신 방식** | 직접 함수 호출 | MCP 프로토콜 |
| **사용 예시** | 학습 목적 | Claude Desktop, 실제 앱 |

**💡 Colab = MCP 학습용, 로컬 = 실제 사용용**

**🚀 다음: 로컬에서 MCP 서버 실행 → Claude Desktop 연동 체험!**

# 6. 🌐 Claude Desktop과 MCP 연동하기

## 🖥️ Claude Desktop에서 MCP 서버 사용

### 📝 설정 파일 위치
- **macOS**: `~/Library/Application Support/Claude/claude_desktop_config.json`
- **Windows**: `%APPDATA%\Claude\claude_desktop_config.json`

### 🔧 기본 설정 예시
```json
{
  "mcpServers": {
    "calculator": {
      "command": "python",
      "args": ["-m", "fastmcp", "serve", "/path/to/calculator_server.py"]
    },
    "textprocessor": {
      "command": "python", 
      "args": ["-m", "fastmcp", "serve", "/path/to/text_server.py"]
    }
  }
}
```

### ⚡ 설정 후 사용법
1. **Claude Desktop 재시작**
2. **채팅에서 바로 사용**: "15와 27을 더해주세요"
3. **자동 도구 호출**: Claude가 MCP 서버의 add 도구를 자동으로 호출

### 🎯 실제 사용 시나리오
```
👤 사용자: "2024년 1월부터 6월까지 매출 데이터를 분석해주세요"
🤖 Claude: [파일 읽기 MCP 도구] → [데이터 분석 MCP 도구] → [차트 생성 MCP 도구]
📊 결과: 완전 자동화된 데이터 분석 리포트
```

# 7. 🚀 실전 MCP 서버 예시들

## 📁 파일 관리 MCP 서버
```python
# file_manager_server.py
from fastmcp import FastMCP
import os
import shutil

file_mcp = FastMCP("FileManager")

@file_mcp.tool()
def list_files(directory: str) -> str:
    """디렉토리의 파일 목록을 반환합니다."""
    files = os.listdir(directory)
    return f"📂 {directory}:\n" + "\n".join(files)

@file_mcp.tool()
def create_folder(path: str) -> str:
    """새 폴더를 생성합니다."""
    os.makedirs(path, exist_ok=True)
    return f"✅ 폴더 생성 완료: {path}"

@file_mcp.tool()
def move_file(source: str, destination: str) -> str:
    """파일을 이동합니다."""
    shutil.move(source, destination)
    return f"📁 파일 이동 완료: {source} → {destination}"
```

## 🌐 웹 스크래핑 MCP 서버
```python
# web_scraper_server.py
from fastmcp import FastMCP
import requests
from bs4 import BeautifulSoup

web_mcp = FastMCP("WebScraper")

@web_mcp.tool()
def get_webpage_title(url: str) -> str:
    """웹페이지의 제목을 가져옵니다."""
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    return f"📄 제목: {soup.title.string}"

@web_mcp.tool()
def extract_links(url: str) -> str:
    """웹페이지의 모든 링크를 추출합니다."""
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    links = [a.get('href') for a in soup.find_all('a', href=True)]
    return f"🔗 링크 {len(links)}개 발견:\n" + "\n".join(links[:10])
```

## 💻 시스템 정보 MCP 서버
```python
# system_info_server.py
from fastmcp import FastMCP
import psutil
import platform

system_mcp = FastMCP("SystemInfo")

@system_mcp.tool()
def get_cpu_usage() -> str:
    """현재 CPU 사용률을 반환합니다."""
    cpu_percent = psutil.cpu_percent(interval=1)
    return f"🖥️ CPU 사용률: {cpu_percent}%"

@system_mcp.tool()
def get_memory_info() -> str:
    """메모리 정보를 반환합니다."""
    memory = psutil.virtual_memory()
    return f"💾 메모리: {memory.percent}% 사용 중 ({memory.used//1024//1024}MB / {memory.total//1024//1024}MB)"

@system_mcp.tool()
def get_system_info() -> str:
    """시스템 기본 정보를 반환합니다."""
    return f"🖥️ OS: {platform.system()} {platform.release()}\n💻 아키텍처: {platform.machine()}"
```

# 8. 🌍 MCP 생태계와 커뮤니티

## 🏢 공식 MCP 서버들 (Anthropic 제공)

| 서버 이름 | 기능 | 사용 사례 |
|-----------|------|----------|
| **filesystem** | 파일 시스템 접근 | 파일 읽기/쓰기, 디렉토리 탐색 |
| **sqlite** | SQLite 데이터베이스 | 데이터 쿼리, 분석 |
| **brave-search** | Brave 검색 API | 웹 검색, 실시간 정보 |
| **github** | GitHub API | 코드 리뷰, 이슈 관리 |
| **google-drive** | Google Drive API | 문서 관리, 협업 |
| **slack** | Slack API | 팀 커뮤니케이션 |

## 🌟 커뮤니티 MCP 서버들

### 📊 데이터 분석
- **pandas-mcp**: 데이터프레임 조작
- **matplotlib-mcp**: 차트 생성
- **jupyter-mcp**: 노트북 실행

### 🤖 AI/ML
- **openai-mcp**: OpenAI API 연동
- **huggingface-mcp**: 모델 추론
- **langchain-mcp**: 체인 실행

### 🌐 웹 서비스
- **twitter-mcp**: 트위터 API
- **youtube-mcp**: YouTube 데이터
- **notion-mcp**: Notion 페이지 관리

## 📈 MCP 생태계의 성장

### 🎯 **2024년 11월 출시 이후**
- ✅ **500+ 커뮤니티 서버** 등장
- ✅ **10만+ 개발자** 사용 중
- ✅ **주요 IDE 지원**: Cursor, VS Code, Zed

### 🚀 **2025년 전망**
- 🔮 **엔터프라이즈 도입** 가속화
- 🔮 **클라우드 서비스 연동** 확대  
- 🔮 **AI 에이전트** 표준 프로토콜로 자리잡음

## 💡 MCP 서버 만들기 베스트 프랙티스

### 🏗️ **구조 설계**
1. **단일 책임**: 하나의 도메인에 집중
2. **명확한 문서화**: 각 도구의 목적과 사용법 상세 기록
3. **에러 처리**: 예외 상황에 대한 적절한 응답

### ⚡ **성능 최적화**
1. **비동기 처리**: I/O 작업은 async/await 사용
2. **캐싱**: 반복적인 요청에 대한 결과 캐싱
3. **배치 처리**: 여러 요청을 묶어서 처리

### 🛡️ **보안 고려사항**
1. **입력 검증**: 모든 입력값에 대한 철저한 검증
2. **권한 관리**: 파일 접근, API 호출 권한 제한
3. **로깅**: 보안 이벤트에 대한 상세 로그

# 9. 🎓 MCP 학습 로드맵과 다음 단계

## 🗺️ MCP 마스터 로드맵

### 📚 **1단계: 기초 이해** (이 노트북에서 완료!)
- ✅ MCP 개념과 필요성 이해
- ✅ FastMCP로 간단한 서버 구축
- ✅ 계산기와 텍스트 처리 도구 구현
- ✅ 로컬 환경 설정 방법 학습

### 🛠️ **2단계: 실전 구현** (다음 목표)
- 📁 **파일 시스템 MCP**: 파일 읽기/쓰기 자동화
- 🌐 **웹 API MCP**: 외부 서비스 연동
- 📊 **데이터 분석 MCP**: 엑셀, CSV 처리 자동화
- 🤖 **AI 모델 MCP**: 로컬 AI 모델 연동

### 🚀 **3단계: 고급 활용** (장기 목표)
- ☁️ **클라우드 배포**: AWS Lambda, Google Cloud Run
- 🔄 **CI/CD 통합**: GitHub Actions 자동화
- 🏢 **엔터프라이즈**: 팀 워크플로우 최적화
- 🌍 **오픈소스 기여**: 커뮤니티 MCP 서버 개발

## 🎯 즉시 시도해볼 수 있는 실습

### 💻 **로컬 환경에서 해보기**
1. **이 노트북의 코드를 .py 파일로 저장**
2. **Claude Desktop에 연결 설정**
3. **실제 채팅에서 "25와 17을 더해주세요" 테스트**

### 🔧 **나만의 MCP 서버 만들기**
```python
# my_custom_server.py
from fastmcp import FastMCP

my_mcp = FastMCP("MyServer")

@my_mcp.tool()
def translate_korean_to_english(text: str) -> str:
    """간단한 한→영 번역 (예시)"""
    translations = {
        "안녕하세요": "Hello",
        "감사합니다": "Thank you",
        "죄송합니다": "Sorry"
    }
    return translations.get(text, f"번역을 찾을 수 없습니다: {text}")

if __name__ == "__main__":
    my_mcp.run()
```

## 📖 추가 학습 자료

### 📚 **공식 문서**
- **MCP 공식 문서**: https://modelcontextprotocol.io
- **FastMCP GitHub**: https://github.com/jlowin/fastmcp  
- **Anthropic MCP Guide**: https://docs.anthropic.com/en/docs/mcp

### 🎥 **동영상 튜토리얼**
- **MCP 소개 영상**: Anthropic 공식 YouTube
- **실전 구현 예시**: 커뮤니티 개발자 채널들

### 💬 **커뮤니티**
- **Discord**: MCP 개발자 커뮤니티
- **Reddit**: r/ModelContextProtocol
- **GitHub Discussions**: 각 MCP 서버 레포지토리

## 🎉 수고하셨습니다!

### 🌟 **이 노트북에서 배운 것들**
- ✨ **MCP의 혁신적 가치**: AI 도구 연결의 새로운 표준
- 🛠️ **FastMCP 활용법**: 간단한 문법으로 강력한 서버 구축
- 🏠 **실제 환경 연동**: Claude Desktop과의 완벽한 통합
- 🌍 **무한한 가능성**: 500+ 커뮤니티 서버의 생태계

### 🚀 **다음 단계**
MCP는 이제 시작입니다. 여러분만의 창의적인 MCP 서버를 만들어서 AI와 현실 세계를 연결해보세요!

**🎯 Happy MCP Coding! 🎯**